# Setup

In [1]:
!pip install -U kaggle-environments
from kaggle_environments import make, evaluate
import numpy as np
#import torch
#import torch.nn.functional as F
from random import choice
#print(np.__version__, torch.__version__)

# Framework

In [0]:
# Constants #

# State #
init, observe = False, None


# Map #
class Direction:

    NORTH = "North"
    SOUTH = "South"
    EAST = "East"
    WEST = "West"
    STATIONARY = None


class Position:
    def __init__(self, x, y, halite=0, ship=None, shipyard=None):
        self.x, self.y, self.halite, self.ship, self.shipyard = x, y, halite, ship, shipyard


# Units #
class Ship:

    IDLE = 0
    MINING = 1
    DEPOSITING = 2
    HUNTING = 3
    CONVERTING = 4

    def __init__(self, unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y
        self.state = self.IDLE
        self.target = None


class Shipyard:
    def __init__(self, unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y


# Observe #
class Observation:
    def __init__(self, config, obs):
        self.turn = 0
        self.CFG = config
        self.SIZE = self.CFG.size
        self.ME = obs.player
        self.PLAYER_N = len(obs.players)

        self.ships = [{} for x in range(self.PLAYER_N)]
        self.shipyards = [{} for x in range(self.PLAYER_N)]
        self.haliteStorage = [0 for x in range(self.PLAYER_N)]
        self.haliteMap = np.empty([self.SIZE, self.SIZE])
        self.board = [[Position(x, y) for x in range(self.SIZE)]
                      for y in range(self.SIZE)]

    def update(self, obs):
        self.PLAYER_N = len(obs.players)
        self.turn += 1

        # Update halite info
        self.haliteMap = np.reshape(
            np.float32(obs.halite), self.haliteMap.shape)
        for x in range(self.CFG.size):
            for y in range(self.CFG.size):
                self.board[x][y].halite = self.haliteMap[x][y]

        # Update player specific info
        raw = obs.players
        for player in range(len(obs.players)):
            self.haliteStorage[player] = obs.players[player][0]
            shipyards = obs.players[player][1]
            ships = obs.players[player][2]
            # Updating state info for seperate units
            tempShips = {}
            tempShipyards = {}
            # Shipyards
            for shipyardID in shipyards.keys():
                x, y = self.decode_pos(shipyards[shipyardID])
                if shipyardID in self.shipyards[player]:
                    self.shipyards[player][shipyardID].x = x
                    self.shipyards[player][shipyardID].x = y
                    tempShipyards[shipyardID] = self.shipyards[player][
                        shipyardID]
                else:
                    tempShipyards[shipyardID] = Shipyard(shipyardID, x, y)
                self.board[x][y].shipyard = tempShipyards[shipyardID]
            self.shipyards[player] = tempShipyards
            # Ships
            for shipID in ships.keys():
                x, y = self.decode_pos(ships[shipID][0])
                if shipID in self.ships[player]:
                    self.ships[player][shipID].x = x
                    self.ships[player][shipID].x = y
                    tempShips[shipID] = self.ships[player][shipID]
                else:
                    tempShips[shipID] = Ship(shipID, x, y)
                tempShips[shipID].halite = ships[shipID][1]
                self.board[x][y].ship = tempShips[shipID]
            self.ships[player] = tempShips

    def decode_pos(self, n):
        return n % self.SIZE, n // self.SIZE

# Calculation and Tool
The code below is used for advanced algorithms

In [ ]:
class Calculator:

    def __init__(self,observe):
        self.OBS = observe

class Toolbox: 

    def __init__(self,observe): 
        self.OBS = observe 

# Agent

In [0]:
def agent(obs, config):
    global init, observe

    action = {}
    
    # init
    if not init:
        observe = Observation(config,obs)
        init = True

    # parse obs
    observe.update(obs)

    # decide

    return action

# Gameplay

In [0]:
env = make("halite", debug=True)
env.run([agent, "random"])
env.render(mode="ipython", width=800, height=600)

In [0]:
import json
print("Configuration:", json.dumps(env.specification.configuration, indent=4, sort_keys=True))
print("Observation:", json.dumps(env.specification.observation, indent=4, sort_keys=True))
print("Action:", json.dumps(env.specification.action, indent=4, sort_keys=True))